In [9]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Activation,Dropout,Flatten,Dense
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from PIL import Image,ImageFilter
import matplotlib.pyplot as plt
import numpy as np
import keras
import cv2
import os
import tensorflow as tf
import time

In [2]:

os.environ["CUDA_VISIBLE_DEVICES"] = "2" #Titan XP
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)
imgSize = 512

In [3]:
#加载检测模型
detect_model = keras.models.load_model("../../model/DETECT_VGG16_256_hid30_epoch10.h5")
detect_model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 30)                983070    
_________________________________________________________________
dropout_3 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 31        
Total params: 15,697,789
Trainable params: 15,697,789
Non-

In [4]:
#加载识别模型
recognize_model = keras.models.load_model("../../model/Recog_RGB_VGG16_512_h50_epoch10.h5")
recognize_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 16, 16, 512)       14714688  
_________________________________________________________________
flatten_13 (Flatten)         (None, 131072)            0         
_________________________________________________________________
dense_25 (Dense)             (None, 50)                6553650   
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 51        
Total params: 21,268,389
Trainable params: 21,268,389
Non-trainable params: 0
_________________________________________________________________


In [5]:
#①分割：用opencv读取进来进行处理，然后进行增强，然后进行分割，分割结果转换为PIL图片
#注意cv2用的是BGR，PIL用的是RGB
def segmentation(origin_img):
    
#   复制边缘进行padding
    #origin_img = cv2.copyMakeBorder(origin_img,100,100,100,100,cv2.BORDER_REPLICATE)   
#     M=np.ones(origin_img.shape,dtype='uint8')*20
#     origin_img=cv2.subtract(origin_img,M)
    origin_img = cv2.copyMakeBorder(origin_img,20,20,20,20,cv2.BORDER_CONSTANT,value=(220,220,220))   
    img = origin_img.copy()
    
    
#   增亮度
#     M=np.ones(img.shape,dtype='uint8')*80
#     img=cv2.add(M,img)

#   转灰度
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #gray = cv2.equalizeHist(gray)
    #gray = cv2.normalize(gray,dst=None,alpha=350,beta=10,norm_type=cv2.NORM_MINMAX)

    
    # #先获取白色背景

    # gray_values = []
    # for i in range(gray.shape[0]):
    #     for j in range(gray.shape[1]):
    #         gray_values.append(gray[i][j])
    # gray_values.sort()
    # mid_value = gray_values[int(len(gray_values)/2)]
    # print(mid_value)

    # for i in range(gray.shape[0]):
    #     for j in range(gray.shape[1]):
    #         if gray[i][j]>mid_value:
    #             gray[i][j] = 255
    #         else:
    #             gray[i][j] = 0

    #转灰度 高糊
    img = cv2.GaussianBlur(img, (9, 9),0) 
    ret,blurred = cv2.threshold(img, 0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    img = cv2.GaussianBlur(img, (9, 9),0) 
    ret,img = cv2.threshold(img, 0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    img = cv2.GaussianBlur(img, (9, 9),0) 
    ret,img = cv2.threshold(img, 0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    #plt.imshow(img)

    # #提取梯度
    # gradX = cv2.Sobel(blurred, ddepth=cv2.CV_32F, dx=1, dy=0)
    # gradY = cv2.Sobel(blurred, ddepth=cv2.CV_32F, dx=0, dy=1)

    # gradient = cv2.subtract(gradX, gradY)
    # gradient = cv2.convertScaleAbs(gradient)

    # plt.imshow(gradient)


    #blurred = cv2.GaussianBlur(gradient, (9, 9),0)
    #(_, img) = cv2.threshold(img, 0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    #plt.imshow(thresh)

    #腐蚀膨胀的核
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    #plt.imshow(closed)

    #腐蚀膨胀
    img = cv2.erode(img, None, iterations=2)
    img = cv2.dilate(img, None, iterations=2)
    #plt.imshow(closed)

    #分割
    (_,contours,_) = cv2.findContours(img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE,)
    #print(len(contours))

    #在原图上裁出分割结果
    segments = []
    best_contours = []
    for c in contours:
        #todo:阈值选取更加合理一点
        if cv2.arcLength(c, True) >500 and cv2.arcLength(c, True) < 5000 :
            (x, y, w, h) = cv2.boundingRect(c)
            crop_img= origin_img[y:y+h, x:x+w]
            #cv2.imwrite('../../data/imgNew/imgFromSeg/'+str(count)+".jpg",crop_img) 
            
            #转为PIL图片
            crop_img = Image.fromarray(cv2.cvtColor(crop_img,cv2.COLOR_BGR2RGB))
            segments.append(crop_img)
            best_contours.append(c)
    return segments,best_contours,origin_img

In [6]:
#②分类
#判断为煤返回m，煤矸石返回s，其他返回n
def recognize(img):
    img = img.filter(ImageFilter.RankFilter(5, 8))
    img2 = img.copy()
    img2 = img2.resize((256,256),Image.ANTIALIAS)
    #img = img.filter(ImageFilter.EDGE_ENHANCE)
    #img = img.filter(ImageFilter.DETAIL)
    isObject = detect_model.predict(np.array(img2).reshape((1,256,256,3))/255)[0]
    #是煤或煤矸石
    if isObject>=0.5:
        img = img.resize((imgSize,imgSize),Image.ANTIALIAS)
        pred_value = recognize_model.predict(np.array(img).reshape((1,imgSize,imgSize,3))/255)[0]
        return 'm' if pred_value<0.5 else 's'
    else:
        return 'n'

In [68]:
#simple test
segments,contours,origin_img = segmentation("../../data/scene/(1).jpg")
for i in range(len(segments)):
    ret = recognize(segments[i])
    (x, y, w, h) = cv2.boundingRect(contours[i])
    if ret == 'm':
        cv2.rectangle(origin_img, (x, y), (x+w, y+h), (255, 0, 0), 8)
    elif ret == 's':
        cv2.rectangle(origin_img, (x, y), (x+w, y+h), (0,0 , 255), 8)
    else:
        cv2.rectangle(origin_img, (x, y), (x+w, y+h), (0,255 ,0 ), 8)
cv2.imwrite("test.jpg",origin_img)    

True

In [27]:
#video process
#读取视频中每一帧，依次处理
cap = cv2.VideoCapture("../../data/video/WIN_20191109_14_51_12_Pro.mp4")
fps = 20
size = (640, 480)
out = cv2.VideoWriter("../../data/video/video_processed/output3.avi", cv2.VideoWriter_fourcc('D', 'I', 'V', 'X') , fps, size)

ret, frame = cap.read()
count = 0
while count<=100 and ret:  
    count+=1
    print("frame",count,end = " ,")
    start = time.clock()
    segments,contours,frame = segmentation(frame)
    seg_end = time.clock()
    print("seg:",seg_end-start,end = ' ')
    for i in range(len(segments)):
        ret = recognize(segments[i])
        (x, y, w, h) = cv2.boundingRect(contours[i])
        if ret == 'm':
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 8)
        elif ret == 's':
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0 , 255), 8)
        else:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255 ,0 ), 8)
    rec_end = time.clock()
    print("recognize:",rec_end-seg_end)
    out.write(frame)
    ret,frame = cap.read()
out.release()
cap.release()

frame 1 ,seg: 0.7389129999974102 

/home/prj201904037/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  del sys.path[0]
/home/prj201904037/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  from ipykernel import kernelapp as app
/home/prj201904037/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


recognize: 11.999979000000167
frame 2 ,seg: 0.6869079999996757 recognize: 12.538712000001397
frame 3 ,seg: 0.5543030000008002 recognize: 12.753002000001288
frame 4 ,seg: 0.6605230000022857 recognize: 12.444503999999142
frame 5 ,seg: 0.6763879999998608 recognize: 12.237742999997863
frame 6 ,seg: 1.0151939999996102 recognize: 12.723698000001605
frame 7 ,seg: 1.0457050000004529 recognize: 12.311934000001202
frame 8 ,seg: 1.0083979999981239 recognize: 12.580312000001868
frame 9 ,seg: 0.959285999997519 recognize: 12.19506100000217
frame 10 ,seg: 0.8703069999974105 recognize: 12.862837000000582
frame 11 ,seg: 0.5603359999986424 recognize: 12.33994900000107
frame 12 ,seg: 0.7584360000000743 recognize: 11.999874000001
frame 13 ,seg: 0.783966999999393 recognize: 12.83930799999871
frame 14 ,seg: 1.086008000002039 recognize: 10.602590999998938
frame 15 ,seg: 0.7178960000019288 recognize: 12.542742999998154
frame 16 ,seg: 0.7144799999987299 recognize: 12.706994000000122
frame 17 ,seg: 0.9650529999